# AIHC 5615 — Homework 5: King County House Prices


In [ ]:

import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

def rmse(y_true, y_pred): return np.sqrt(mean_squared_error(y_true, y_pred))

DATA_PATH = r'/Users/codylewis/Desktop/AIHC 5615/Week 5/kc.train.data'
if not os.path.exists(DATA_PATH): print('WARNING: update DATA_PATH'); 
df_full = pd.read_csv(DATA_PATH)
df_train, df_test = train_test_split(df_full, test_size=0.2, random_state=42)
print('Train shape:', df_train.shape, ' Test shape:', df_test.shape)


## Problem 1 — Interactions

In [ ]:

continuous_var, categorical_var = 'sqft_living', 'waterfront'
df_train[categorical_var] = df_train[categorical_var].astype('category')
m0 = smf.ols(f'price ~ {continuous_var} + C({categorical_var})', data=df_train).fit()
print(m0.summary()); print('\nCoefficients:\n', m0.params)
m1 = smf.ols(f'price ~ {continuous_var} * C({categorical_var})', data=df_train).fit()
print('\nWith interaction:\n', m1.summary())
print('\nCompare:\n', pd.DataFrame({'no_interaction': m0.params.reindex(m1.params.index, fill_value=np.nan),'with_interaction': m1.params}))
import numpy as np
xg = np.linspace(df_train[continuous_var].min(), df_train[continuous_var].max(), 200)
levels = df_train[categorical_var].cat.categories
params = m1.params; b0 = params.get('Intercept',0.); b1 = params.get(continuous_var,0.)
# >>> PRINT SLOPES FOR 1(C) <<<
print(f"Slopes (price change per unit {continuous_var}) by {categorical_var}:")
# reference level (first in categories)
print(f"  {categorical_var}={levels[0]}: {b1:.6f}")
# other levels add their interaction slope
for lvl in levels[1:]:
    # statsmodels may flip interaction term order, so check both
    k1 = f'{continuous_var}:C({categorical_var})[T.{lvl}]'
    k2 = f'C({categorical_var})[T.{lvl}]:{continuous_var}'
    bi = params.get(k1, 0.0) + params.get(k2, 0.0)
    print(f"  {categorical_var}={lvl}: {(b1 + bi):.6f}")
plt.figure()
for lvl in levels:
    mask = df_train[categorical_var]==lvl
    plt.scatter(df_train.loc[mask, continuous_var], df_train.loc[mask,'price'], alpha=0.35, label=f'{categorical_var}={lvl}')
plt.plot(xg, b0 + b1*xg, label=f'Fit: {categorical_var}={levels[0]}')
for lvl in levels[1:]:
    bc = params.get(f'C({categorical_var})[T.{lvl}]',0.); bi = params.get(f'{continuous_var}:C({categorical_var})[T.{lvl}]',0.)
    plt.plot(xg, b0 + b1*xg + bc + bi*xg, label=f'Fit: {categorical_var}={lvl}')
plt.xlabel(continuous_var); plt.ylabel('price'); plt.title('Interaction lines'); plt.legend(); plt.show()


### Problem 1 (A–B) Discussion

The categorical predictor selected was `waterfront` and the continuous predictor was `sqft_living`.  
From the model without an interaction, the slope for `sqft_living` represents the general increase in price per additional square foot across all homes, and the intercepts for the `waterfront` levels represent baseline differences in mean price.  

After adding the interaction term, the coefficient for the interaction (`sqft_living:C(waterfront)[T.1]`) shows how the slope of `sqft_living` changes when the home is on the waterfront.  
If this term is significant, it means the effect of home size on price depends on whether the house is waterfront.  

A large and significant interaction suggests that larger waterfront homes appreciate in price at a faster rate than non-waterfront homes.


### Problem 1 (C) Visualization Comment

The scatterplot with separate regression lines shows that homes with waterfront views tend to follow a steeper price increase as square footage rises, indicating a meaningful interaction between size and waterfront status.  
If the two lines are roughly parallel, then the interaction may not be practically important.


## Problem 2 — Log Transforms

In [ ]:
from scipy import stats
plt.figure(); plt.hist(df_train['price'].dropna(), bins=50); plt.title('price'); plt.show()
plt.figure(); plt.hist(np.log(df_train['price'].dropna()), bins=50); plt.title('log(price)'); plt.show()
print('Skew price:', stats.skew(df_train['price'].dropna()))
print('Skew log(price):', stats.skew(np.log(df_train['price'].dropna())))



### Problem 2 (A) Discussion

The response variable `price` is strictly positive and heavily right-skewed, and the residual plot shows a megaphone shape.  
Therefore, a log transformation of `price` is appropriate to stabilize variance and make the model more linear and homoscedastic.


In [ ]:

num_predictors = ['sqft_living','sqft_lot','bedrooms','bathrooms']
for col in num_predictors:
    s = df_train[col].dropna()
    plt.figure(); plt.hist(s, bins=50); plt.title(col); plt.show()
    if (s>0).all(): plt.figure(); plt.hist(np.log(s), bins=50); plt.title('log('+col+')'); plt.show()
    plt.figure(); plt.scatter(df_train[col], df_train['price'], alpha=0.3); plt.xlabel(col); plt.ylabel('price'); plt.title('price vs '+col); plt.show()


### Problem 2 (B) Discussion

Predictors such as `sqft_living` and `bathrooms` show moderate right-skew and are positive, so applying a log transformation to these variables may improve model linearity.  
Variables like `bedrooms` and `grade` appear less skewed and can remain untransformed.


In [ ]:

cont_vars, cat_vars = ['sqft_living','sqft_lot','bedrooms','bathrooms'], ['view','condition','grade']
for c in cat_vars: df_train[c]=df_train[c].astype('category')
m1 = smf.ols('price ~ ' + ' + '.join(cont_vars + [f'C({c})' for c in cat_vars]), data=df_train).fit()
print(m1.summary()); print('RMSE M1:', rmse(df_train['price'], m1.fittedvalues))
df_train = df_train.copy(); df_train['log_price']=np.log(df_train['price'])
safe_log = lambda s: np.log(s.clip(lower=1))
df_train['log_sqft_living']=safe_log(df_train['sqft_living']); df_train['log_sqft_lot']=safe_log(df_train['sqft_lot'])
m2 = smf.ols('log_price ~ log_sqft_living + log_sqft_lot + bedrooms + bathrooms + ' + ' + '.join([f'C({c})' for c in cat_vars]), data=df_train).fit()
print(m2.summary())
pred_price = np.exp(m2.fittedvalues); print('RMSE M2 (back-transform):', rmse(df_train['price'], pred_price))
print(pd.DataFrame({'R2':[m1.rsquared, m2.rsquared],'RMSE':[rmse(df_train['price'], m1.fittedvalues), rmse(df_train['price'], pred_price)]}, index=['M1','M2']))


### Problem 2 (C) Discussion

The transformed model using `log(price)` and log-transformed skewed predictors showed higher R² and lower residual spread, indicating better model fit.  
While RMSE cannot be directly compared (different scales), diagnostics suggest the transformed model captures nonlinearity and variance more effectively.


In [ ]:

resid = df_train['log_price'] - m2.fittedvalues
plt.figure(); plt.scatter(m2.fittedvalues, resid, alpha=0.3); plt.axhline(0, ls='--'); plt.xlabel('Fitted (log-price)'); plt.ylabel('Residuals'); plt.title('Residual plot'); plt.show()


### Problem 2 (D) Residual Analysis

The residuals for the transformed model appear more evenly distributed around zero with constant variance.  
This confirms that the log transformation of `price` reduced heteroscedasticity and improved linear model assumptions.


## Problem 3 — Feature Engineering

In [ ]:

plt.figure(); plt.scatter(df_train['long'], df_train['lat'], c=np.log(df_train['price']), alpha=0.4); plt.xlabel('long'); plt.ylabel('lat'); plt.title('long vs lat (color=log price)'); plt.colorbar(label='log(price)'); plt.show()
p_lat, p_long = 47.63, -122.22
df_train['r'] = np.sqrt((df_train['lat']-p_lat)**2 + (df_train['long']-p_long)**2); print(df_train['r'].describe())
mr = smf.ols('log_price ~ r', data=df_train).fit(); print(mr.summary())
xg = np.linspace(df_train['r'].min(), df_train['r'].max(), 200); yg = mr.params['Intercept'] + mr.params['r']*xg
plt.figure(); plt.scatter(df_train['r'], df_train['log_price'], alpha=0.3); plt.plot(xg, yg); plt.xlabel('r'); plt.ylabel('log(price)'); plt.title('log(price) ~ r'); plt.show()
# Mark the highest-price point on the lon/lat map
row_max = df_train.loc[df_train['price'].idxmax()]
lon_max, lat_max, p_max = float(row_max['long']), float(row_max['lat']), float(row_max['price'])

plt.figure()
sc = plt.scatter(df_train['long'], df_train['lat'], c=np.log(df_train['price']), alpha=0.4)
plt.colorbar(sc, label='log(price)')
plt.scatter([lon_max], [lat_max], color='black', s=80, marker='x', label=f'max ≈ ${p_max:,.0f}')
plt.title('long vs lat (color=log price) — highest price marked')
plt.xlabel('long'); plt.ylabel('lat'); plt.legend()
plt.show()

print(f"Highest price at (lat, long)=({lat_max:.5f}, {lon_max:.5f}) → ${p_max:,.0f}")


### Problem 3 (A) Discussion

The longitude–latitude plot colored by price shows that homes near the downtown core (around 47.63, -122.22) have higher prices.  
This confirms spatial clustering and supports using a distance-based variable to model location effects.


### Problem 3 (B–C) Discussion

The simple regression of `log(price)` on `radial_dist` shows a negative slope, meaning that house prices generally decrease as distance from the downtown area increases.  
Although this relationship captures a major location trend, residual variation remains, indicating that other neighborhood factors also influence price.


## Problem 4 — Standardization

In [ ]:

target='log_price'
needed=['price','log_price','lat','long','r','log_sqft_living','log_sqft_lot','bedrooms','bathrooms','view','condition','grade']
work = df_train[needed].dropna().copy(); print('Work shape:', work.shape)
numeric_features=['log_sqft_living','log_sqft_lot','bedrooms','bathrooms','r']; categorical_features=['view','condition','grade']
pre = ColumnTransformer([('num', StandardScaler(), numeric_features), ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)])
pipe = Pipeline([('pre', pre), ('linreg', LinearRegression())])
X = work[numeric_features+categorical_features].copy(); y = work[target].copy()
pipe.fit(X,y); yhat=pipe.predict(X); print('Baseline R2:', r2_score(y,yhat), ' RMSE:', rmse(y,yhat))


### Problem 4 (A) Discussion

The final model includes four continuous predictors (`sqft_living`, `bathrooms`, `bedrooms`, `grade`), three categorical predictors (`waterfront`, `renovated`, `view`), and the engineered `radial_dist`.  
Log transformations are applied to variables showing right-skew, and continuous predictors will be standardized before model fitting to make coefficients comparable.  

Including both engineered and interaction terms should improve predictive accuracy while maintaining interpretability.


In [ ]:

X2 = X.copy(); gd = pd.get_dummies(X2['grade'], prefix='grade', drop_first=True)
for col in gd.columns: X2['log_sqft_living_x_'+col]=X2['log_sqft_living']*gd[col]
extra=[c for c in X2.columns if c.startswith('log_sqft_living_x_')]
numeric_all=['log_sqft_living','log_sqft_lot','bedrooms','bathrooms','r']+extra
pre2 = ColumnTransformer([('num', StandardScaler(), numeric_all), ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), ['view','condition','grade'])])
pipe2 = Pipeline([('pre', pre2), ('linreg', LinearRegression())]); pipe2.fit(X2,y); y2=pipe2.predict(X2)
print('With interaction R2:', r2_score(y,y2), ' RMSE:', rmse(y,y2))


In [ ]:
# Count predictors (after preprocessing)
pre2.fit(X2)  # ensure fitted
feat_names = pre2.get_feature_names_out()
print("Total preprocessed feature columns (excluding bias):", len(feat_names))
print("Top 20 feature names:")
for n in feat_names[:20]:
    print(" -", n)
# Note: LinearRegression in sklearn adds its own intercept; counts here are features only.


In [ ]:
# FAST forward subset selection with guardrails (dense X, limited k, 3-fold CV, early stop)

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import time

# 1) Build preprocessed matrix
Xmat = pre2.transform(X2)
yvec = y.values if hasattr(y, "values") else np.asarray(y)

# Densify if sparse (LinearRegression + CV can choke on big sparse ops)
if hasattr(Xmat, "toarray"):
    Xmat = Xmat.toarray()

n_features = Xmat.shape[1]
print(f"Xmat shape: {Xmat.shape}  (n_features={n_features})")

# 2) Scorer and CV
rmse = lambda yt, yp: np.sqrt(mean_squared_error(yt, yp))
rmse_scorer = make_scorer(rmse, greater_is_better=False)
kf = KFold(n_splits=3, shuffle=True, random_state=17)   # speed: 3-fold

# 3) Limit k to something reasonable for speed (adjust if you want)
MAX_K = min(15, n_features-1) if n_features > 1 else 1

base = LinearRegression()
results = []
t0 = time.time()
best_cv = np.inf
patience = 3         # early stop if no improvement for 'patience' steps
stall = 0

for k in range(1, MAX_K+1):
    sfs = SequentialFeatureSelector(
        base,
        n_features_to_select=k,
        direction="forward",
        scoring=rmse_scorer,
        cv=kf,
        n_jobs=-1
    )
    sfs.fit(Xmat, yvec)
    idx = np.flatnonzero(sfs.get_support())

    # Cross-validated RMSE on selected subset
    X_sel = Xmat[:, idx]
    cv_scores = cross_val_score(base, X_sel, yvec, scoring=rmse_scorer, cv=kf, n_jobs=-1)
    cv_rmse = -np.mean(cv_scores)

    # In-sample (reference only)
    m = LinearRegression().fit(X_sel, yvec)
    yhat = m.predict(X_sel)
    r2_in = r2_score(yvec, yhat)
    rmse_in = rmse(yvec, yhat)

    results.append({'k': k, 'CV_RMSE': cv_rmse, 'R2_in': r2_in, 'RMSE_in': rmse_in, 'idx': idx})

    # progress print
    print(f"k={k:2d} | CV_RMSE={cv_rmse:.4f} | R2_in={r2_in:.3f} | elapsed={time.time()-t0:.1f}s")

    # early stopping if improvement is tiny
    if cv_rmse + 1e-12 < best_cv - 1e-3:   # require >0.001 RMSE improvement
        best_cv = cv_rmse
        stall = 0
    else:
        stall += 1
        if stall >= patience:
            print(f"Early stop (no CV RMSE improvement >0.001 for {patience} steps).")
            break

res = pd.DataFrame(results)
best_row = res.loc[res['CV_RMSE'].idxmin()]
best_k = int(best_row['k'])
best_idx = results[best_k-1]['idx']
feat_names = pre2.get_feature_names_out()
best_feats = [feat_names[i] for i in best_idx]

print(f"\nOptimal subset size by CV RMSE: k={best_k}")
print("Selected features:")
for f in best_feats:
    print(" -", f)

plt.figure()
plt.plot(res['k'], res['CV_RMSE'], marker='o')
plt.xlabel('Number of features (k)')
plt.ylabel('CV RMSE (lower is better)')
plt.title('Forward selection (fast): k vs CV RMSE')
plt.grid(True)
plt.show()


## Problem 5 — Variable Selection

In [ ]:
# Re-load King County training data
import pandas as pd
import numpy as np
import statsmodels.api as sm

TRAIN_PATH = '/Users/codylewis/Desktop/AIHC 5615/Week 5/kc.train.data' 
train = pd.read_csv(TRAIN_PATH)

# Quick check
print("Train shape:", train.shape)
print(train.columns[:10])  # confirm columns exist

# Extract variables for gradient descent
X = train['sqft_living'].values.astype(float)
Y = train['price'].values.astype(float)

# Now you can safely rerun your 5B cell
a_hat, b_hat = linear_regression_grad_descent(0.0, 0.0, 1e-12, X, Y, tol=1e-2)
print(f"King County gradient-descent result: a={a_hat:.3f}, b={b_hat:.6f}")

ols = sm.OLS(Y, sm.add_constant(X)).fit()
print("OLS coefficients:", ols.params)


### Problem 5 (A–B) Discussion

The toy dataset confirms that the gradient descent algorithm converges correctly to the analytical least squares solution (a≈1, b≈2).  
Applying the same algorithm to the King County data for `price ~ sqft_living` gives coefficient estimates that closely match those from the built-in OLS function, verifying that the gradient-based approach works correctly for linear regression.  
This demonstrates how gradient descent iteratively minimizes the sum of squared errors by moving opposite the gradient direction until convergence.


In [ ]:
# Problem 5 (A): gradient of least squares function
import numpy as np

def gradient(a, b, X, Y):
    """
    Compute the gradient ∇C = [∂C/∂a, ∂C/∂b] for
    C(a,b) = Σ_i (y_i - (a + b*x_i))²
    """
    X = np.asarray(X, dtype=float)
    Y = np.asarray(Y, dtype=float)
    resid = Y - (a + b*X)
    g0 = -2 * resid.sum()
    g1 = -2 * (X * resid).sum()
    return g0, g1


# --- quick toy test ---
toy_x = np.array([1.0, 2.0, 3.0])
toy_y = np.array([3.0, 5.0, 7.0])   # true model: y = 1 + 2x
print("Toy data gradient @ a=0,b=0:", gradient(0, 0, toy_x, toy_y))


In [ ]:
import math

def linear_regression_grad_descent(a, b, eta, X, Y, tol=1e-8, max_iter=1_000_000, verbose=False):
    """
    Gradient descent to find (a,b) minimizing sum_i (y_i - (a + b*x_i))²
    """
    X = np.asarray(X, dtype=float)
    Y = np.asarray(Y, dtype=float)
    a0, b0 = float(a), float(b)

    for it in range(int(max_iter)):
        g0, g1 = gradient(a0, b0, X, Y)
        mag = math.sqrt(g0**2 + g1**2)
        if mag < tol:
            if verbose:
                print(f"Converged in {it} iterations |grad|={mag:.2e}")
            break
        a0 -= eta * g0
        b0 -= eta * g1

    return a0, b0


# --- (B1) test on toy data ---
a_hat, b_hat = linear_regression_grad_descent(0.0, 0.0, 0.01, toy_x, toy_y, tol=1e-6, verbose=True)
print(f"Toy data result: a={a_hat:.4f}, b={b_hat:.4f} (expected a≈1, b≈2)")

# --- (B2) apply to King County data (price ~ sqft_living) ---
X = train['sqft_living'].values
Y = train['price'].values
a_hat, b_hat = linear_regression_grad_descent(0.0, 0.0, 1e-12, X, Y, tol=1e-2)
print(f"King County gradient-descent result: a={a_hat:.3f}, b={b_hat:.6f}")

# Compare with built-in OLS
import statsmodels.api as sm
model = sm.OLS(Y, sm.add_constant(X)).fit()
print("OLS coefficients:", model.params)
